# 💻 Romlig kobling

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/GMGI221-2024/forelesninger/blob/main/05_romlig_kobling.ipynb)

*Romlige koblinger* er operasjoner som kombinerer data fra to eller flere romlige datasett
basert på deres geometriske relasjon. Vi har allerede fått
kjennskap til to spesifikke tilfeller av romlige koblinger: [Punkt-i-polygon
spørringer](#05_romlig_utvelgelse) og [Intersect](#02_intersect). 

Romlige koblingsoperasjoner krever to inngangsparametere: *predikatet*, dvs. den
geometriske betingelsen som må oppfylles mellom to geometrier, og
*koblings-typen*: om bare rader med matchende geometrier beholdes, eller alle radene i en
inputtabell, eller alle radene fra begge tabellene.

*Geopandas* (ved hjelp av `shapely` for å implementere geometriske forhold) [støtter et
standard sett med geometriske
predikater](https://geopandas.org/en/stable/docs/user_guide/mergingdata.html#binary-predicate-joins),
som ligner de som de fleste GIS analyseverktøy og progammer bruker:

- intersects
- contains
- within
- touches
- crosses
- overlaps

Geometriske predikater uttrykkes som verb, så de har en intuitiv
betydning. Se [shapely bruker
manual](https://shapely.readthedocs.io/en/stable/manual.html#binary-predicates)
for en detaljert beskrivelse av hvert geometrisk predikat.


:::{admonition} Binære geometriske predikater
:class: hint

Shapely støtter flere *binære geometriske predikater* enn geopandas implementerer
for romlige koblinger. Hva er de? Kan de uttrykkes ved å kombinere de
som allerede er implementerte?
:::

Når det gjelder *koblings-typen*, implementerer geopandas tre forskjellige alternativer:

- *left*: behold alle poster av *left* data-rammen, fyll med tomme verdier hvis
  ingen match, behold *left* geometrikolonne
- *right*: behold alle poster av *left* data-rammen, fyll med tomme verdier hvis
  ingen match, behold *right* geometrikolonne
- *inner*: behold bare poster av matchende poster, behold *left* geometrikolonne

![Romlige koblinger](https://pygis.io/_images/join_types.jpg)

:::{tip}
[PyGIS
boken](https://pygis.io/docs/e_spatial_joins.html) har en god oversikt over
romlige predikater og koblings-typer med forklarende tegninger.
:::


---


## Last inn inngangsdata

Som et praktisk eksempel, la oss finne befolkningstettheten på hver av
adressene fra [tidligere i denne leksjonen](#05_geokoding_i_geopandas), ved å kombinere
datasettet med data fra et befolkningsnett.

Befolkningsnettdata er tilgjengelig fra [SSB](https://kart.ssb.no/).

In [1]:
import pathlib 
NOTEBOOK_PATH = pathlib.Path().resolve()
DATA_MAPPE = NOTEBOOK_PATH / "data"

In [19]:
import geopandas

adresser = geopandas.read_file(DATA_MAPPE / "oslo_adresser" / )

befolkningsnett = geopandas.read_file(DATA_MAPPE / "ssb_rutenett" / )

Til slutt, beregn befolkningstettheten ved å dele antall innbyggere
av hver rutenettcelle med arealet i km²:

:::{admonition} Kodestil: store tall, operatorer i flerlinjeuttrykk
:class: tip

Hvis du trenger å bruke veldig store tall, for eksempel, i eksempelet ovenfor, *1
million* for å konvertere m² til km², kan du bruke understreker (`_`) som
tusen-separatorer. Python vil behandle en sekvens av tall
sammenvevd med understreker som en vanlig numerisk verdi.
[Du kan bruke samme syntaks for å gruppere
tall](https://peps.python.org/pep-0515/) etter en annen logikk, for eksempel,
for å gruppere heksadesimale eller binære verdier i grupper av fire.

I tilfelle et uttrykk, for eksempel, en matematisk formel, sprer seg over
flere linjer, anses det for å være god kodestil å plassere en operator i
begynnelsen av en ny linje, i stedet for å la den være en hale i den forrige linjen. Dette
anses som mer lesbart, som forklart i [PEP-8 styling
retningslinjer](https://peps.python.org/pep-0008/#should-a-line-break-before-or-after-a-binary-operator)
:::


---

## Join inputlagene


Nå er vi klare til å utføre den romlige joinen mellom de to lagene.
Husk: målet er å finne befolkningstettheten rundt hver av adressepunktene. Vi ønsker å knytte befolkningstetthetsinformasjon fra
`befolkningsnett`-polygonlaget til `adresser`-punktlaget, avhengig av
om **punktet er innenfor polygonet**. Under denne operasjonen ønsker vi å
**beholde geometriene til punktlaget**.

Før vi kan fortsette med join-operasjonen, må vi sørge for at de to
lagene er i samme crs:

In [ ]:
assert adresser.crs == befolkningsnett.crs, "CRS er ikke identiske"

De deler ikke samme CRS, la oss reprojisere befolkningsnettet, siden vi ønsker å joine infomasjon fra befolkningsnettet inn i adresse-laget vårt:

Nå er vi klare til å utføre den faktiske romlige joinen ved hjelp av
[`geopandas.GeoDataFrame.sjoin()`](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.sjoin.html)
metoden. Husk, vi ønsker å bruke et *within* geometrisk predikat og beholde
punktlagets geometrier (i eksempelet nedenfor er *left* dataframmen).

Det ser flott ut! Vi har nå et adresse-datasett med befolkningstetthetsinformasjon knyttet til det. 


---


Som en siste oppgave, la oss se på hvordan du kan plotte data ved hjelp av en *gradert*
kartografisk visualiseringsstil. 

`geopandas.GeoDataFrame.plot()` metoden kan variere kartfargene avhengig av en kolonnes verdier ved å sende inn navnet som et navngitt argument `column`. I tillegg til det, aksepterer metoden mange argumenter for å påvirke stilen på kartet. Blant dem er `scheme` og `cmap` som definerer [kategoriseringsskjemaet](https://geopandas.org/en/stable/gallery/choropleths.html), og [fargekartet](https://matplotlib.org/stable/tutorials/colors/colormaps.html) som brukes. Mange flere argumenter sendes gjennom til `matplotlib`, som `markersize` for å sette størrelsen på punktsymboler, og `facecolor` for å sette fargen på polygonområder. For å tegne en legend, sett `legend` til `True`, for å sette størrelsen på figuren, angi inn en tuple (med verdier i tommer) som `figsize`.

---

Vi kan bruke de samme argumentene for å plotte et befolkningstetthetskart ved hjelp av
hele `befolkningsnett` datasettet:

---


Til slutt, kan vi lagre output-dataframmen til en fil: